
# Assignment 5

Implement a code for aspect mining and topic modeling

# Topic Modeling

In [1]:
doc1 = "She loves to write short stories in the local coffee shop."
doc2 = "My Bill spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Nisha and her friends danced happily."
doc5 = "She won't be attending the Met Gala this year."

doc_complete = [doc1, doc2, doc3, doc4, doc5]

In [2]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = ' '.join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join([ch for ch in stop_free if ch not in exclude])
    normalized = ' '.join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
doc_clean = [clean(doc).split() for doc in doc_complete]

In [12]:
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [13]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics = 3, id2word = dictionary, passes=50)

In [14]:
print(ldamodel.print_topics(num_topics=3, num_words=4))


[(0, '0.056*"may" + 0.056*"increased" + 0.056*"suggest" + 0.056*"blood"'), (1, '0.062*"write" + 0.062*"story" + 0.062*"local" + 0.062*"love"'), (2, '0.068*"driving" + 0.068*"sister" + 0.068*"lot" + 0.068*"dance"')]


# Aspect Mining

In [18]:
!pip install stanza
import stanza
#stanza.download('en')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 4.0 MB/s 
     |████████████████████████████████| 240 kB 44.6 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=f6a8525f68493b17139cb083d5ea5920f6c34fd55bc52ef16e9d5d843e1fd4d4
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [19]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [22]:
!pip install stanfordnlp
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanfordnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 158 kB 4.0 MB/s 


In [23]:
txt = "Battery life is ok, camera is bad but The Sound Quality is great."

In [24]:
txt = txt.lower()
sentList = nltk.sent_tokenize(txt)

In [25]:
for line in sentList:
    txt_list = nltk.word_tokenize(line)
    taggedList = nltk.pos_tag(txt_list)
print(taggedList)

[('battery', 'NN'), ('life', 'NN'), ('is', 'VBZ'), ('ok', 'JJ'), (',', ','), ('camera', 'NN'), ('is', 'VBZ'), ('bad', 'JJ'), ('but', 'CC'), ('the', 'DT'), ('sound', 'JJ'), ('quality', 'NN'), ('is', 'VBZ'), ('great', 'JJ'), ('.', '.')]


In [26]:
newwordList = []
flag = 0
for i in range(0,len(taggedList)-1):
    if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
        newwordList.append(taggedList[i][0]+taggedList[i+1][0])
        flag=1
    else:
        if(flag==1):
            flag=0
            continue
        newwordList.append(taggedList[i][0])
        if(i==len(taggedList)-2):
            newwordList.append(taggedList[i+1][0])
finaltxt = ' '.join(word for word in newwordList)
print(finaltxt)

batterylife is ok , camera is bad but the sound quality is great .


In [27]:
stop_words = set(stopwords.words('english'))
new_txt_list = nltk.word_tokenize(finaltxt)
wordsList = [w for w in new_txt_list if not w in stop_words]
taggedList = nltk.pos_tag(wordsList)

In [28]:
nlp = stanza.Pipeline()
doc = nlp(finaltxt)
dep_node = []
for dep_edge in doc.sentences[0].dependencies:
    dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
for i in range(0, len(dep_node)):
    if(int(dep_node[i][1]) != 0):
        dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
print(dep_node)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[['batterylife', 'ok', 'nsubj'], ['is', 'ok', 'cop'], ['ok', 0, 'root'], [',', 'ok', 'punct'], ['camera', 'bad', 'nsubj'], ['is', 'bad', 'cop'], ['bad', 'ok', 'conj'], ['but', 'great', 'cc'], ['the', 'quality', 'det'], ['sound', 'quality', 'compound'], ['quality', 'great', 'nsubj'], ['is', 'great', 'cop'], ['great', 'ok', 'conj'], ['.', 'ok', 'punct']]


In [29]:
featureList = []
categories = []
totalfeatureList = []
for i in taggedList:
    if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
        featureList.append(list(i))
        totalfeatureList.append(list(i)) # stores all the features for every sentence
        categories.append(i[0])
print(featureList)
print(categories)

[['batterylife', 'NN'], ['ok', 'NN'], ['camera', 'NN'], ['bad', 'JJ'], ['sound', 'JJ'], ['quality', 'NN'], ['great', 'JJ']]
['batterylife', 'ok', 'camera', 'bad', 'sound', 'quality', 'great']


In [30]:
fcluster = []
for i in featureList:
    filist = []
    for j in dep_node:
        if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
            if(j[0]==i[0]):
                filist.append(j[1])
            else:
                filist.append(j[0])
    fcluster.append([i[0], filist])
print(fcluster)

[['batterylife', ['ok']], ['ok', ['batterylife']], ['camera', ['bad']], ['bad', ['camera']], ['sound', ['quality']], ['quality', ['sound', 'great']], ['great', ['quality']]]


In [31]:
finalcluster = []
dic = {}
for i in featureList:
    dic[i[0]] = i[1]
for i in fcluster:
    if(dic[i[0]]=='NN'):
        finalcluster.append(i)
print(finalcluster)

[['batterylife', ['ok']], ['ok', ['batterylife']], ['camera', ['bad']], ['quality', ['sound', 'great']]]
